<a href="https://colab.research.google.com/github/SkandeshRanganathan/Skandesh-s-portfolio/blob/main/imageprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
data_dir = "/content/sample_data/tumour"

In [ ]:
dataset = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    image_size=(128, 128),  # Resize images to 128x128
    batch_size=32,         # Number of images per batch
    shuffle=True           # Shuffle the data
)

Found 3734 files belonging to 2 classes.


In [ ]:
dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [ ]:
images = []
labels = []
for images_batch, labels_batch in dataset:
    images.extend(images_batch.numpy())
    labels.extend(labels_batch.numpy())

images = np.array(images)
labels = np.array(labels)

# Now you can use train_test_split
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.2, random_state=42
)

# Create tf.data.Dataset objects from the split data if needed
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Get the number of classes from your dataset
num_classes = len(np.unique(train_labels)) # Assuming 'train_labels' contains class labels

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')  # One output unit per class
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels)) # Use the correct variable names


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 117s 1s/step - accuracy: 0.6451 - loss: 334.6939 - val_accuracy: 0.8608 - val_loss: 5.8130
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 137s 1s/step - accuracy: 0.8718 - loss: 4.7045 - val_accuracy: 0.9237 - val_loss: 2.1066
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 140s 1s/step - accuracy: 0.9651 - loss: 0.7775 - val_accuracy: 0.9264 - val_loss: 2.3979
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 160s 1s/step - accuracy: 0.9746 - loss: 0.4727 - val_accuracy: 0.9505 - val_loss: 1.8765
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 133s 1s/step - accuracy: 0.9769 - loss: 0.4458 - val_accuracy: 0.9130 - val_loss: 2.5486
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 116s 972ms/step - accuracy: 0.9564 - loss: 1.2598 - val_accuracy: 0.9545 - val_loss: 1.4702
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 133s 884ms/step - accuracy: 0.9950 - loss: 0.0760 - val_accuracy: 0.9531 - val_loss: 1.4469
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 80s 855ms/step - accuracy: 0.9957 - loss: 0.0526 - val_accuracy: 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load data from directory
# Replace 'data_dir' with the actual path to your training data directory
train_generator = train_datagen.flow_from_directory(
    data_dir,  # Changed from 'path/to/train' to data_dir
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Replace 'data_dir' with the path to your test data.
# If the test data is in a subdirectory of data_dir, like 'data_dir/test', use that.
test_generator = test_datagen.flow_from_directory(
    data_dir,  # Changed from 'path/to/test' to data_dir for demonstration
    target_size=(128, 128),
    batch_size=32,
    class_mode='sparse'
)

# Train the model
model.fit(train_generator, epochs=10, validation_data=test_generator)


Found 4182 images belonging to 3 classes.
Found 4182 images belonging to 3 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 166s 1s/step - accuracy: 0.5101 - loss: 2.6249 - val_accuracy: 0.7083 - val_loss: 0.6019
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 211s 1s/step - accuracy: 0.6803 - loss: 0.5966 - val_accuracy: 0.7327 - val_loss: 0.5600
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.6969 - loss: 0.5949 - val_accuracy: 0.6779 - val_loss: 0.6559
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 165s 1s/step - accuracy: 0.7352 - loss: 0.5370 - val_accuracy: 0.7420 - val_loss: 0.5464
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 168s 1s/step - accuracy: 0.7477 - loss: 0.5274 - val_accuracy: 0.7977 - val_loss: 0.4608
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 201s 1s/step - accuracy: 0.7493 - loss: 0.5191 - val_accuracy: 0.7580 - val_loss: 0.5631
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 164s 1s/step - accuracy: 0.7717 - loss: 0.4824 - val_accuracy: 0.7697 - val_loss: 0.5140
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 174s 1s/step - accuracy: 0.7737 - loss: 0.4778 - val_accuracy: 0.805